In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx
import dbase_tools

# Import API key
from api_keys import api_key
single_city_url = "http://api.openweathermap.org/data/2.5/weather?"

## Bokeh imports

In [2]:
#%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import DatetimeTickFormatter
##import pandas as pd
#import numpy as np
from bokeh.models.tools import HoverTool
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [5]:
city_name = "portland"
city_id = "5746545"
city_units = "metric"
covid = "New_York_Times/covid-19-state-level-data.csv"

In [6]:
# Create City/location and transport Classes
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class State(Base):
    __tablename__ = 'location'
    id = Column(Integer, primary_key=True)
    city_name = Column(String(255))
    date = Column(String(255))
    cases = Column(Integer)
    deaths = Column(Integer)
    new_cases = Column(Integer)
    new_deaths = Column(Integer)

In [7]:
engine = create_engine("sqlite:///coviddata.sqlite")
conn = engine.connect()
Base.metadata.create_all(engine)

In [9]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [7]:
covid_df = pd.read_csv(covid)

In [8]:
covid_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
1824,2020-04-04,Virginia,51,2407,52
1825,2020-04-04,Washington,53,7498,318
1826,2020-04-04,West Virginia,54,282,2
1827,2020-04-04,Wisconsin,55,2112,60


## data cleaning

In [9]:
covcopy = covid_df.copy()

In [10]:
state_list = covid_df['state'].unique()

In [11]:
state_list

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands'], dtype=object)

## Isolate regions by state

In [12]:
all_states = dbase_tools.group_states(covid_df,state_list)

In [13]:
all_states.head()

,date,state,fips,cases,deaths,new_deaths,new_cases
0,2020-01-21,Washington,53,1,0,0,1
1,2020-01-22,Washington,53,1,0,0,0
2,2020-01-23,Washington,53,1,0,0,0
3,2020-01-24,Illinois,17,1,0,0,0
4,2020-01-24,Washington,53,1,0,0,0


In [14]:
region_list_state = all_states.loc[all_states['state'] == "New York"]

In [15]:
region_list_state.head(5)

,date,state,fips,cases,deaths,new_deaths,new_cases
246,2020-03-01,New York,36,1,0,0,0
261,2020-03-02,New York,36,1,0,0,1
276,2020-03-03,New York,36,2,0,4,92
293,2020-03-04,New York,36,11,0,0,0
313,2020-03-05,New York,36,22,0,0,0


### start here for state_data_sort function.

# Create object to be stored in database

In [16]:
dbase_tools.prep_for_db(all_states)

dog_list = session.query(Dog)
for doggy in dog_list:
    print(doggy.name)

In [10]:
date_list = session.query(State)
for date in date_list:
    print(date.cases)

1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
2
1
1
1
2
1
1
1
2
2
1
1
3
2
1
1
3
2
1
1
1
6
2
1
1
1
6
2
1
1
1
6
2
1
1
1
6
2
1
1
1
1
6
2
1
1
1
1
6
2
1
1
1
1
6
2
1
1
1
1
6
2
1
1
1
1
7
2
1
1
1
1
7
2
1
1
1
1
7
2
1
1
1
1
1
7
2
1
2
1
1
1
7
2
1
2
1
1
1
7
2
1
2
1
1
1
7
2
1
2
1
1
1
7
2
1
10
2
1
1
1
7
2
1
10
2
1
1
1
7
2
1
10
2
1
1
1
8
2
1
11
2
1
1
1
9
2
1
11
4
1
1
1
9
2
1
11
4
1
1
1
9
2
1
11
4
1
1
1
11
2
1
12
10
5
1
1
11
2
1
13
10
1
5
1
1
26
2
1
13
10
1
5
1
1
26
2
1
13
10
1
5
1
1
27
2
1
13
1
11
1
7
1
1
28
3
1
13
1
11
1
10
1
1
33
2
3
1
13
1
2
2
11
1
17
1
1
38
2
2
4
2
13
1
1
2
2
11
1
23
1
2
45
3
2
4
2
13
2
2
1
2
2
11
1
32
1
2
55
3
2
4
2
13
2
1
11
1
2
2
12
1
47
1
2
67
2
4
2
5
3
3
13
2
2
2
22
1
2
2
1
16
1
75
1
3
81
8
7
3
1
6
1
1
3
8
1
14
2
2
4
44
2
1
2
2
3
2
1
19
3
86
1
5
100
8
1
12
7
1
6
1
1
1
3
13
1
1
14
2
4
4
89
2
1
6
4
3
2
1
19
3
1
1
110
1
5
112
8
1
1
13
11
2
7
2
3
1
4
5
28
2
1
16
4
4
6
106
2
1
13
6
3
6
3
23
3
1
2
141
1
6
172
12
2
4
14
17
2
11
4
8
1
5
1
6
41
2
1
16
4
4
11
142
7
3
1
13
10
3
7
4
24
3
1
5
179
2


In [19]:
ref_date = []

for date in region_list_country['date']:
    ref_date.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

date_groups = []

for date in indy_location_moving[0]['date']:
    date_groups_1.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

In [20]:
date_cases = []

for date in ref_date:
    date_cases.append(str(date))

# pass plot data as well as hovertool data in to plot.
source_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['cases'],
            desc=date_cases,
        )
    )

In [21]:
# pass plot data as well as hovertool data in to plot.
source_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['deaths'],
            desc=date_cases,
        )
    )

In [22]:
# pass plot data as well as hovertool data in to plot.
source_new_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['new_deaths'],
            desc=date_cases,
        )
    )

In [23]:
# pass plot data as well as hovertool data in to plot.
source_new_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['new_cases'],
            desc=date_cases,
        )
    )

In [24]:
# configure plot tools including the hover tool / tooltips.  Displays date / time of individual data points when hovering 
#over individual points.
TOOLSCCD = [HoverTool(tooltips=[
            ("cvd_date", "@desc"),("cases","$y")],
                  formatters={'DateTime': 'datetime'}),'wheel_zoom','save','crosshair','pan','reset']

In [25]:
cid = figure(title = f"Values for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cid.xaxis.axis_label = 'Event Date'
cid.yaxis.axis_label = 'COVID Total Cases'
# Setup x axis for using Datetime formatting.
cid.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cid.xaxis.major_label_orientation = pi/4
cid.circle('x', 'y', line_color="navy", fill_color="red", fill_alpha=0.2, size=10,source=source_cases)


GlyphRenderer(id='1045', ...)

In [26]:
cix = figure(title = f"Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cix.xaxis.axis_label = 'Event Date'
cix.yaxis.axis_label = 'COVID Total Deaths'
# Setup x axis for using Datetime formatting.
cix.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cix.xaxis.major_label_orientation = pi/4
cix.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_deaths)


GlyphRenderer(id='1087', ...)

In [27]:
cin = figure(title = f"New Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cin.xaxis.axis_label = 'Event Date'
cin.yaxis.axis_label = 'COVID New Deaths'
# Setup x axis for using Datetime formatting.
cin.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cin.xaxis.major_label_orientation = pi/4
cin.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_new_deaths)

GlyphRenderer(id='1129', ...)

In [28]:
cinc = figure(title = f"New Cases for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cinc.xaxis.axis_label = 'Event Date'
cinc.yaxis.axis_label = 'COVID New Cases'
# Setup x axis for using Datetime formatting.
cinc.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cinc.xaxis.major_label_orientation = pi/4
cinc.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_new_cases)

GlyphRenderer(id='1171', ...)

In [29]:
show(cid)

In [30]:
show(cix)

In [31]:
show(cin)

In [32]:
show(cinc)